In [145]:
import os
import openai
from langchain_openai import OpenAIEmbeddings, ChatOpenAI  # Correct imports from langchain_openai
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, create_retrieval_chain  # Latest import method for retrieval chains
from langchain.agents import create_structured_chat_agent, Tool, AgentExecutor # Updated agent creation method

from langchain_core.output_parsers.json import SimpleJsonOutputParser
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [76]:
# Certifique-se de importar a biblioteca OpenAI

# Defina o caminho completo para o arquivo api.txt
api_path = r"C:\Users\Victor Broering\pycode\api.txt"

# Verifique se o arquivo existe
if os.path.exists(api_path):
    # Abra o arquivo e leia o conteúdo (API key)
    with open(api_path, 'r') as file:
        api_key = file.read().strip()
        print("API carregada com sucesso!")
else:
    print(f"O arquivo {api_path} não foi encontrado.")

# Atribuir a chave da API ao OpenAI
openai.api_key = api_key


API carregada com sucesso!


In [188]:
import re

# Função para dividir o texto com base em cabeçalhos HTML
def dividir_por_secoes(texto):
    # Usar regex para dividir o texto em pedaços com base nos títulos <h1>, <h2>, <h3>
    padroes = re.compile(r"(<h[1-3]>.*?</h[1-3]>)", re.DOTALL)
    secoes = padroes.split(texto)

    # Agrupar os cabeçalhos com seus conteúdos correspondentes
    chunks = []
    for i in range(0, len(secoes)-1, 2):
        chunk = secoes[i] + secoes[i+1]
        chunks.append(chunk)
    
    return chunks

# Carregar o conteúdo do arquivo txt
with open(txt_file, 'r', encoding='utf-8') as file:
    raw_text = file.read()

# Dividir o texto em seções com base nos cabeçalhos HTML
chunks = dividir_por_secoes(raw_text)

# Verificar o número de pedaços
print(f"Dividido em {len(chunks)} pedaços.")

# Criar o armazenamento vetorial usando FAISS
vector_store = FAISS.from_texts(chunks, embeddings)

# Salvar o armazenamento vetorial na pasta 'assets'
vector_store.save_local("./assets/vector_store_acao_alimentos")
print("Armazenamento vetorial salvo com sucesso na pasta 'assets'.")


Dividido em 20 pedaços.
Armazenamento vetorial salvo com sucesso na pasta 'assets'.


In [189]:
from langchain.chains import RetrievalQA  # Use RetrievalQA direto

# Carregar o vetor de armazenamento FAISS da pasta 'assets'
vector_store = FAISS.load_local(
    "./assets/vector_store_acao_alimentos", 
    embeddings, 
    allow_dangerous_deserialization=True
)

# Configurar o modelo de linguagem com ChatOpenAI
llm = ChatOpenAI(model_name='gpt-4', temperature=0.5, openai_api_key=api_key)

# Criar a cadeia de Pergunta e Resposta usando RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(),
    chain_type="stuff"
)




In [190]:
# TESTE COM APENAS TREXOS DO EXEMPLO

from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# Definir a ferramenta para utilizar a vector store
def buscar_exemplo(narrativa):
    # Usa o vector store para buscar o exemplo mais similar à narrativa completa
    resultado = vector_store.similarity_search(narrativa, k=1)  # Retorna o exemplo mais relevante
    if resultado:
        exemplo_inspirado = resultado[0].page_content
        
        # Aqui, você pode gerar uma nova narrativa baseada no estilo do exemplo encontrado
        nova_narrativa = gerar_texto_inspirado(narrativa, exemplo_inspirado)
        return nova_narrativa
    else:
        return "Nenhum exemplo encontrado."
    
def gerar_texto_inspirado(narrativa, exemplo_inspirado):
        # Aqui você pode usar um modelo de linguagem (como GPT-4) para gerar um novo texto
        # inspirado no exemplo de estilo
        prompt = f"""
        Mantendo-se fiel ao estilo de redação do exemplo a seguir, redija a seção 'Dos Fatos' de uma petição de ação de alimentos com base nos acontecimentos narrados do caso:
        
        Exemplo: {exemplo_inspirado}
        
        Narrativa do caso: {narrativa}
        
        Resposta (em tom jurídico, formal e claro):
        """
        
        # Chamar o modelo de linguagem para gerar o novo texto
        resposta = llm(prompt)  # Supondo que você tenha configurado o LLM como o GPT-4
        return resposta
tools = [
    Tool(
        name="BuscarExemploJuridico",  # Nome ajustado para seguir o padrão permitido
        func=buscar_exemplo,
        description="Busca um template do tópico dos fatos de uma ação de alimentos relevante na vector store."
    )
]

# Definir o prompt

# Definir o prompt correto com templates, incluindo o 'agent_scratchpad'
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "1. Você é um assistente jurídico que irá incorporar o estilo de redação do Autor que gerou o conteúdo na vector store. "
            "2. Você precisa se relembrar dos conceitos atualizados do Direito das Famílias Brasileiro moderno, exemplo: o uso da palavra 'menor' é discriminatório."
        ),
        HumanMessagePromptTemplate.from_template("{input}"),
        # Incluir o `agent_scratchpad` no final, como requerido pelo agente
        HumanMessagePromptTemplate.from_template("{agent_scratchpad}")
    ]
)

# Criar o agente usando a ferramenta e o prompt
agent = create_tool_calling_agent(llm, tools, prompt)


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



In [192]:
def buscar_exemplo_por_secao(narrativa, secao="DOS FATOS"):
    # Usa o vector store para buscar o exemplo mais similar à narrativa completa
    resultado = vector_store.similarity_search(narrativa, k=1)  # Retorna o exemplo mais relevante
    if resultado:
        exemplo_inspirado = resultado[0].page_content
        
        # Ver o conteúdo completo do resultado
        print("Exemplo completo encontrado:\n", exemplo_inspirado)
        
        # Agora tentar buscar a seção dentro desse exemplo
        inicio_secao = exemplo_inspirado.lower().find(secao.lower())
        
        if inicio_secao != -1:
            fim_secao = exemplo_inspirado.find("\n", inicio_secao + len(secao))
            if fim_secao == -1:
                fim_secao = len(exemplo_inspirado)
            secao_texto = exemplo_inspirado[inicio_secao:fim_secao]
            print(f"Seção '{secao}' encontrada:\n", secao_texto)
            return secao_texto
        else:
            print(f"Seção '{secao}' não encontrada no exemplo.")
            return "Seção não encontrada."
    else:
        print("Nenhum exemplo encontrado.")
        return "Nenhum exemplo encontrado."



# Função para contar tokens
def contar_tokens(texto, modelo="gpt-4"):
    # Selecionar o codificador de tokens com base no modelo
    encoding = tiktoken.encoding_for_model(modelo)
    
    # Tokenizar o texto
    tokens = encoding.encode(texto)
    
    # Retornar a contagem de tokens
    return len(tokens)

# Exemplo de uso:
texto_consulta = "Quais são os 'DOS FATOS' da peça jurídica sobre alimentos?"
tokens_usados = contar_tokens(texto_consulta)




query = "Quais são os 'DOS FATOS' da peça jurídica sobre alimentos?"
resposta = qa_chain.run(query)
print("Resposta do modelo:\n", resposta)
print(f"Tokens usados na consulta: {tokens_usados}")

# Isso vai imprimir apenas a seção "DOS FATOS" do exemplo encontrado:
# Seção 'DOS FATOS' encontrada: <conteúdo da seção>


Resposta do modelo:
 Desculpe, mas o texto fornecido não inclui uma seção específica chamada "DOS FATOS". No entanto, o contexto geral do texto indica que o réu tem a responsabilidade legal de prover o sustento de sua filha, apesar de alegar dificuldades financeiras devido à formação de uma nova família. A autora, que está em uma situação de vulnerabilidade, precisa de um reajuste no valor dos alimentos devidos pelo réu, pois o valor atual não é suficiente para cobrir todas as necessidades da criança. Além disso, o réu tem uma renda considerável de suas atividades comerciais e investimentos, mas frequentemente oculta a real extensão de seus ganhos. Portanto, é solicitado um aumento na contribuição financeira do réu.
Tokens usados na consulta: 19


In [191]:

# Narrativa de exemplo para testar a função
narrativa_teste = f"""
Lucas, de 6 anos, está propondo uma ação de alimentos, guarda e convivência em face de Bruno Silva, o pai de Pedro.

Bruno e Júlia mantiveram um relacionamento por 4 anos, mas se separaram há cerca de 1 ano. Desde a separação, Pedro reside exclusivamente com sua mãe, Júlia, em Florianópolis/SC, enquanto o pai, Bruno, vive em Palhoça/SC.

Bruno tem visitado o filho raramente e alega que suas despesas mensais o impedem de contribuir de maneira mais significativa para o sustento de Pedro. Ele tem oferecido apenas R$ 300,00 por mês, alegando estar com dificuldades financeiras. Júlia, por outro lado, argumenta que Bruno possui um bom emprego como gerente de uma empresa de comércio eletrônico, com uma renda mensal em torno de R$ 15.000,00.

Júlia solicita a guarda unilateral de Pedro, alegando que Bruno não se interessa em cuidar do filho de maneira ativa e que a falta de regularidade nas visitas está afetando o relacionamento entre pai e filho. Ela também requer que seja fixada uma pensão alimentícia de 25% dos rendimentos de Bruno e que a convivência seja regulamentada para que Pedro tenha uma relação saudável com o pai."""

# Chamar a função buscar_exemplo passando a narrativa_teste
exemplo_encontrado = buscar_exemplo(narrativa_teste)

# Exibir o resultado
print(f"Exemplo encontrado: {exemplo_encontrado}")


Exemplo encontrado: content='<h3>Dos Fatos</h3>\n\nO suplicante, Lucas, menor impúbere, representado por sua genitora, Sra. Júlia, vem propor a presente ação de alimentos, guarda e convivência em face do Sr. Bruno Silva, seu genitor.\n\nO relacionamento entre a Sra. Júlia e o Sr. Bruno perdurou por 4 anos, findando-se há aproximadamente 1 ano. Desde então, Lucas reside com sua genitora na cidade de Florianópolis/SC, enquanto o réu reside em Palhoça/SC.\n\nO réu tem se mostrado negligente no que tange à sua responsabilidade parental, realizando visitas esporádicas ao infante e contribuindo de maneira insuficiente para seu sustento. Alega que suas despesas mensais o impedem de contribuir de maneira mais significativa, oferecendo apenas a quantia de R$ 300,00 mensais, sob a justificativa de dificuldades financeiras.\n\nContudo, tal alegação não se sustenta diante do fato de que o réu ocupa o cargo de gerente em uma empresa de comércio eletrônico, possuindo uma renda mensal estimada em tor

In [198]:


# Função para buscar o exemplo e isolar a seção "Dos Fatos"
def buscar_exemplo_por_secao(narrativa, secao="DOS FATOS"):
    # Usa o vector store para buscar o exemplo mais similar à narrativa completa
    resultado = vector_store.similarity_search(narrativa, k=1)  # Retorna o exemplo mais relevante
    if resultado:
        exemplo_inspirado = resultado[0].page_content
        
        # Exibe o conteúdo completo encontrado para referência
        print("Exemplo completo encontrado:\n", exemplo_inspirado)

        # Buscar o título da peça no formato <h1>
        inicio_titulo = exemplo_inspirado.lower().find("<h1>")
        fim_titulo = exemplo_inspirado.lower().find("</h1>", inicio_titulo)
        titulo = exemplo_inspirado[inicio_titulo + len("<h1>"):fim_titulo].strip()

              # Buscar pela seção especificada no formato <h2> ou <h1>
            for secao in secoes_possiveis:
                inicio_secao = exemplo_inspirado.lower().find(f"<h2>{secao.lower()}</h2>")
            
            if inicio_secao == -1:
                inicio_secao = exemplo_inspirado.lower().find(f"<h1>{secao.lower()}</h1>")

            if inicio_secao != -1:
                # Procurar o próximo <h2> ou <h1> para definir o fim da seção
                fim_secao_h2 = exemplo_inspirado.find("<h2>", inicio_secao + len(secao))
                fim_secao_h1 = exemplo_inspirado.find("<h1>", inicio_secao + len(secao))

                # Definir o fim da seção como o mais próximo dos dois
                fim_secao = min(fim_secao_h2 if fim_secao_h2 != -1 else len(exemplo_inspirado),
                                fim_secao_h1 if fim_secao_h1 != -1 else len(exemplo_inspirado))

                # Extrair o texto da seção
                secao_texto = exemplo_inspirado[inicio_secao:fim_secao]
                print(f"Seção '{secao}' encontrada:\n", secao_texto)
                return secao_texto
        
        # Se nenhuma seção for encontrada, retornar o exemplo completo como fallback
        print("Nenhuma seção específica encontrada. Usando o conteúdo completo.")
        return exemplo_inspirado
    else:
        print("Nenhum exemplo encontrado.")
        return "Nenhum exemplo encontrado."

def limitar_tamanho_dos_chunks(chunks, chunk_size=1000):
    chunks_limitados = []
    for chunk in chunks:
        while len(chunk) > chunk_size:
            # Se o chunk for muito grande, dividir em partes menores
            chunks_limitados.append(chunk[:chunk_size])
            chunk = chunk[chunk_size:]
        chunks_limitados.append(chunk)
    return chunks_limitados

# Dividir o texto por seções
chunks = dividir_por_secoes(raw_text)

# Limitar o tamanho dos chunks
chunks_limitados = limitar_tamanho_dos_chunks(chunks, chunk_size=800)

# Verificar o número de pedaços após limitar o tamanho
print(f"Dividido em {len(chunks_limitados)} pedaços.")

from bs4 import BeautifulSoup

# Carregar o conteúdo do arquivo txt
with open(txt_file, 'r', encoding='utf-8') as file:
    raw_text = file.read()

# Usar BeautifulSoup para analisar o conteúdo HTML
soup = BeautifulSoup(raw_text, "html.parser")

# Extrair todas as seções <h1>, <h2>, <h3> com seus conteúdos
sections = soup.find_all(['h1', 'h2', 'h3'])

# Construir chunks com base nas seções
chunks = []
for section in sections:
    content = section.get_text() + "\n"
    next_sibling = section.find_next_sibling(string=True)
    while next_sibling and next_sibling.name not in ['h1', 'h2', 'h3']:
        content += next_sibling.get_text()
        next_sibling = next_sibling.find_next_sibling(string=True)
    chunks.append(content)


# Verificar o número de pedaços
print(f"Dividido em {len(chunks)} pedaços.")

# Criar o armazenamento vetorial usando FAISS
vector_store = FAISS.from_texts(chunks, embeddings)

# Salvar o armazenamento vetorial na pasta 'assets'
vector_store.save_local("./assets/vector_store_acao_alimentos")
print("Armazenamento vetorial salvo com sucesso na pasta 'assets'.")

# Função para contar tokens
def contar_tokens(texto, modelo="gpt-4"):
    # Selecionar o codificador de tokens com base no modelo
    encoding = tiktoken.encoding_for_model(modelo)

    # Tokenizar o texto
    tokens = encoding.encode(texto)

    # Retornar a contagem de tokens
    return len(tokens)

# Função para gerar o texto inspirado no exemplo
def gerar_texto_inspirado(narrativa, exemplo_inspirado):
    # Criar o prompt baseado no exemplo encontrado
    prompt = f"""
    Mantendo-se fiel ao estilo de redação do exemplo a seguir, redija a seção 'Dos Fatos' de uma petição de ação de alimentos com base nos acontecimentos narrados do caso:

    Exemplo: {exemplo_inspirado}

    Narrativa do caso: {narrativa}

    Resposta (em tom jurídico, formal e claro):
    """

    # Chamar o modelo de linguagem para gerar o novo texto
    resposta = llm(prompt)  # Supondo que você tenha configurado o LLM como o GPT-4
    return resposta

# Configuração das ferramentas
tools = [
    Tool(
        name="BuscarExemploJuridico",  # Nome ajustado para seguir o padrão permitido
        func=buscar_exemplo,
        description="Busca um template do tópico dos fatos de uma ação de alimentos relevante na vector store."
    )
]

# Definir o prompt correto com templates, incluindo o 'agent_scratchpad'
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "1. Você é um assistente jurídico que irá incorporar o estilo de redação do Autor que gerou o conteúdo na vector store. "
            "2. Você precisa se relembrar dos conceitos atualizados do Direito das Famílias Brasileiro moderno, exemplo: o uso da palavra 'menor' é discriminatório."
        ),
        HumanMessagePromptTemplate.from_template("{input}"),
        # Incluir o `agent_scratchpad` no final, como requerido pelo agente
        HumanMessagePromptTemplate.from_template("{agent_scratchpad}")
    ]
)

# Criar o agente usando a ferramenta e o prompt
agent = create_tool_calling_agent(llm, tools, prompt)

# Criar o executor do agente
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)




Dividido em 24 pedaços.
Dividido em 20 pedaços.
Armazenamento vetorial salvo com sucesso na pasta 'assets'.


In [200]:
narrativa = f"""
Lucas, de 6 anos, está propondo uma ação de alimentos, guarda e convivência em face de Bruno Silva, o pai de Pedro.

Bruno e Júlia mantiveram um relacionamento por 4 anos, mas se separaram há cerca de 1 ano. Desde a separação, Pedro reside exclusivamente com sua mãe, Júlia, em Florianópolis/SC, enquanto o pai, Bruno, vive em Palhoça/SC.

Bruno tem visitado o filho raramente e alega que suas despesas mensais o impedem de contribuir de maneira mais significativa para o sustento de Pedro. Ele tem oferecido apenas R$ 300,00 por mês, alegando estar com dificuldades financeiras. Júlia, por outro lado, argumenta que Bruno possui um bom emprego como gerente de uma empresa de comércio eletrônico, com uma renda mensal em torno de R$ 15.000,00.

Júlia solicita a guarda unilateral de Pedro, alegando que Bruno não se interessa em cuidar do filho de maneira ativa e que a falta de regularidade nas visitas está afetando o relacionamento entre pai e filho. Ela também requer que seja fixada uma pensão alimentícia de 25% dos rendimentos de Bruno e que a convivência seja regulamentada para que Pedro tenha uma relação saudável com o pai.
"""

# Testar a busca pela seção "DOS FATOS"
secao_dos_fatos = buscar_exemplo_por_secao(narrativa, secao="DOS FATOS")

if secao_dos_fatos != "Seção não encontrada.":
    # Contar tokens da seção e da narrativa
    tokens_secao = contar_tokens(secao_dos_fatos)
    tokens_narrativa = contar_tokens(narrativa)

    print(f"Tokens usados na seção: {tokens_secao}")
    print(f"Tokens usados na narrativa: {tokens_narrativa}")

    # Gerar o texto inspirado
    texto_inspirado = gerar_texto_inspirado(narrativa, secao_dos_fatos)
    print("Texto inspirado gerado:\n", texto_inspirado)
else:
    print("Seção 'DOS FATOS' não encontrada no exemplo.")


Exemplo completo encontrado:
 Ação de alimentos, guarda e convivência com guarda unilateral, restrição de convivência e presença de violência domestica




A autora, filha legítima de [nome do pai] e [nome da mãe], nasceu em [data de nascimento], na cidade de Adrianópolis-PR. Desde o nascimento, a filiação foi prontamente reconhecida por ambos os pais, conforme certidão de nascimento anexa. A genitora da autora sempre assumiu a responsabilidade principal pelos cuidados e criação da criança. O réu, embora presente fisicamente, demonstrava ausência emocional, além de comportamentos violentos em diversas ocasiões.


Os pais da autora iniciaram o relacionamento no ambiente de trabalho em Adrianópolis-PR, em outubro de 2014. A convivência se estabeleceu em abril de 2015, sob pressão de chefes da fábrica onde ambos trabalhavam. Inicialmente, o relacionamento era pautado na cooperação, com divisão das tarefas e preocupação conjunta com o bem-estar familiar. Entretanto, a demissão da genitora 